In [1]:
#导库层
import numpy as np
import pandas as pd
import datetime
from dateutil.relativedelta import relativedelta
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from decimal import Decimal, ROUND_HALF_UP
import math
import os

In [2]:
#数据截止至2023年8月25日，交易费用截止至2023年9月20日
#读取数据层
AHInfo = pd.read_csv("data/A+HInfo.csv", encoding='ANSI')
ARHab = pd.read_csv("data/ARHab.csv", encoding='gbk')
HAInfo = pd.read_csv("data/H+AInfo.csv", encoding='utf_8_sig')
HRHabByRMB = pd.read_csv("data/HRHabByRMB.csv", encoding='gbk')
HKDCNY = pd.read_csv("data/HKDCNY.EX.csv", encoding='ANSI')
AHInfoSH = AHInfo.query('证券代码.str.contains("SH")', engine='python')
AHInfoSZ = AHInfo.query('证券代码.str.contains("SZ")', engine='python')
#重新设置索引
ARHab = ARHab.set_index(['date'])
HRHabByRMB = HRHabByRMB.set_index(['date'])
HKDCNY = HKDCNY.set_index(['时间'])
HKDCNY = HKDCNY.drop(columns=['代码', '简称'])
#设置索引格式
ARHab.index = pd.to_datetime(ARHab.index, format='%Y-%m-%d')
HRHabByRMB.index = pd.to_datetime(HRHabByRMB.index, format='%Y-%m-%d')
HKDCNY.index = pd.to_datetime(HKDCNY.index, format='%Y-%m-%d')

d:\py\lib\site-packages\IPython\core\interactiveshell.py:3361: DtypeWarning: Columns (5,32,35,50,74,75,98,99) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
d:\py\lib\site-packages\IPython\core\interactiveshell.py:3361: DtypeWarning: Columns (62) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [81]:
#函数层
#数据选择函数
#输入A股证券代码，查询A股和H股的对应序列
def DataSelect(AID):
    #AID:A股代码
    HID = AHInfo.loc[AHInfo["证券代码"] == AID]["同公司港股代码"]
    HID = HID.iloc[0]
    data = pd.merge(ARHab[AID].to_frame(), HRHabByRMB[HID].to_frame(), left_index=True, right_index=True, how='outer')
    return data


#去除空值行
def DataClean(data):
    #data:为要处理的dataframe
    data = data.dropna()
    return data


#线性回归，返回截距与系数
def OLS(x, y):
    #x:解释变量，y:解释变量
    x = sm.add_constant(x)
    model = sm.OLS(y, x)
    OLSresults = model.fit()
    constant = OLSresults.params[0]
    slope = OLSresults.params[1]
    return [constant, slope]


#判断是否能成为配对，返回是否为配对，1为是，0为否;截距;系数;残差的标准差
def IfPair(data, T0, tf, CD):
    #data为要检验的数据，start为起始时间，精确到日，length为时间跨度，单位为月份，CD为置信度
    T0 = datetime.datetime.strptime(T0, "%Y-%m-%d")
    end = T0 + relativedelta(months=tf)
    temp = data.loc[T0:end]
    temp.iloc[:, 0] = np.log(data.iloc[:, 0])
    temp.iloc[:, 1] = np.log(data.iloc[:, 1])
    #进行线性回归
    x = temp.iloc[:, 0]
    y = temp.iloc[:, 1]
    parameter = OLS(x, y)
    re = y - x * parameter[1] - parameter[0]
    std = re.std()
    #ADF检验
    ADFresultC = adfuller(re, regression='c')
    if ADFresultC[1] < CD:
        return [1, parameter[0], parameter[1], std]
    ADFresultCT = adfuller(re, regression='ct')
    if ADFresultCT[1] < CD:
        return [1, parameter[0], parameter[1], std]
    ADFresultCTT = adfuller(re, regression='ctt')
    if ADFresultCTT[1] < CD:
        return [1, parameter[0], parameter[1], std]
    ADFresultCNC = adfuller(re, regression='nc')
    if ADFresultCNC[1] < CD:
        return [1, parameter[0], parameter[1], std]
    return [0, parameter[0], parameter[1], std]


#计算A股交易费用
def CalculateATF(IsS, SNV):
    # IsS:是否是卖方,SNV：股票净值
    # A股佣金比例（ACR）：0.0003,不满5元按5元收取;印花税(SD)：向卖方收取0.00005；过户费(TransferF)：0.00001；交易所规费(EF)：0.0000687；
    ACR = 0.0003
    SD = 0.00005
    TransferF = 0.00001
    EF = 0.0000687
    if IsS == True:
        ACRF = Decimal(SNV * ACR).quantize(Decimal("0.00")) if Decimal(SNV * ACR).quantize(Decimal("0.00")) > 5 else 5
        SDF = Decimal(SNV * SD).quantize(Decimal("0.00"))
        TransferFF = Decimal(SNV * TransferF).quantize(Decimal("0.00"))
        EFF = Decimal(SNV * EF).quantize(Decimal("0.00"))
        ATF = ACRF + SDF + TransferFF + EFF
        return ATF
    else:
        ACRF = Decimal(SNV * ACR).quantize(Decimal("0.00")) if Decimal(SNV * ACR).quantize(Decimal("0.00")) > 5 else 5
        TransferFF = Decimal(SNV * TransferF).quantize(Decimal("0.00"))
        EFF = Decimal(SNV * EF).quantize(Decimal("0.00"))
        ATF = ACRF + TransferFF + EFF
        return ATF


#计算H股交易费用
def CalculateHTF(IsS, SNV, ExchangeRate):
    #SNV：股票净值
    # 港股佣金比例(ACR)：0.0025，最低收费100港币；证监会交易征费(SFCTL)：0.000027;财务汇报局交易征费(FRCTL)：0.0000015;交易费(TF):0.0000565;H股印花税(SD)：0.0013，不足一元亦作一元计;
    # 转手纸印花税(SDOTP):5港币，卖方负责缴付；过户费用(TransferF):2.50港币，由买方支付
    ACR = 0.0025
    SFCTL = 0.000027
    FRCTL = 0.0000015
    TF = 0.0000565
    SD = 0.0013
    SDOTP = 5
    TransferF = 2.50
    if IsS == True:
        ACRF = Decimal(SNV * ACR).quantize(Decimal("0.00")) if Decimal(SNV * ACR).quantize(Decimal("0.00")) > Decimal(
            100 * ExchangeRate).quantize(Decimal("0.00")) else Decimal(100 * ExchangeRate).quantize(Decimal("0.00"))
        SFCTLF = Decimal(SNV * SFCTL).quantize(Decimal("0.00"))
        FRCTLF = Decimal(SNV * FRCTL).quantize(Decimal("0.00"))
        TFF = Decimal(SNV * TF).quantize(Decimal("0.00"))
        SDF = math.ceil(SNV * SD)
        SDOTPF = Decimal(SDOTP * ExchangeRate).quantize(Decimal("0.00"))
        ATF = ACRF + SFCTLF + FRCTLF + TFF + SDF + SDOTPF
        return ATF
    else:
        ACRF = Decimal(SNV * ACR).quantize(Decimal("0.00")) if Decimal(SNV * ACR).quantize(Decimal("0.00")) > Decimal(
            100 * ExchangeRate).quantize(Decimal("0.00")) else Decimal(100 * ExchangeRate).quantize(Decimal("0.00"))
        SFCTLF = Decimal(SNV * SFCTL).quantize(Decimal("0.00"))
        FRCTLF = Decimal(SNV * FRCTL).quantize(Decimal("0.00"))
        TFF = Decimal(SNV * TF).quantize(Decimal("0.00"))
        SDF = math.ceil(SNV * SD)
        TransferFF = Decimal(TransferF * ExchangeRate).quantize(Decimal("0.00"))
        ATF = ACRF + SFCTLF + FRCTLF + TFF + SDF + TransferFF
        return ATF


#分配初始金额函数,返回值为数组:A股数量;港股数量
def AllotmentAmount(All, r, ExchangeRate, PA, PH):
    # All:初始资金;r:比例系数，为在判断是否为配对的函数中的比例系数;DateTime:分配初始金额的日期;date:股票的价格数据
    #A股佣金比例（ACR）：0.0003,不满5元按5元收取;印花税(SD)：向卖方收取0.00005；过户费(TransferF)：0.00001；交易所规费(EF)：0.0000687；
    # 港股佣金比例(ACR)：0.0025，最低收费100港币；证监会交易征费(SFCTL)：0.000027;财务汇报局交易征费(FRCTL)：0.0000015;交易费(TF):0.0000565;H股印花税(SD)：0.0013，不足一元亦作一元计;
    # 转手纸印花税(SDOTP):5港币，卖方负责缴付；过户费用(TransferF):2.50港币，由买方支付
    T = 2.5 * ExchangeRate  #过户费用
    rA = 0.0003787  #A股交易费用的比例部分
    rH = 0.003885  #H股交易费用的比例部分
    rA1 = 0.0000787  #A股去除佣金部分的比例系数
    ACRA = 0.0003  #A股佣金比例（ACR）
    ACRFA = 5  #A股佣金固定数值
    rH1 = 0.001385  #H股去除佣金部分的比例系数
    ACRFH = 100 * ExchangeRate  #H股佣金固定数值
    ACRH = 0.0025  #H股佣金比例（ACR）
    nH = (All - T) / (PA * r + PA * r * rA + PH + PH * rH)
    nA = r * nH
    if (PA * nA * ACRA) > 5 and (PH * nH * ACRH) > (100 * ExchangeRate):
        nA = math.floor(nA)
        nH = math.floor(nH)
        return [nA, nH]
    nH = (All - T - ACRFA) / (PA * r + PA * r * rA1 + PH + PH * rH)
    nA = r * nH
    if (PA * nA * ACRA) <= 5 and (PH * nH * ACRH) > (100 * ExchangeRate):
        nA = math.floor(nA)
        nH = math.floor(nH)
        return [nA, nH]
    nH = (All - T - ACRFH) / (PA * r + PA * r * rA + PH + PH * rH1)
    nA = r * nH
    if (PA * nA * ACRA) > 5 and (PH * nH * ACRH) <= (100 * ExchangeRate):
        nA = math.floor(nA)
        nH = math.floor(nH)
        return [nA, nH]
    nH = (All - T - ACRFA - ACRFH) / (PA * r + PA * r * rA1 + PH + PH * rH1)
    nA = r * nH
    if (PA * nA * ACRA) <= 5 and (PH * nH * ACRH) <= (100 * ExchangeRate):
        if nA > 0:
            nA = math.floor(nA)
            nH = math.floor(nH)
            return [nA, nH]
        else:
            return [0, 0]


#购买A股函数，返回这笔资金能购买的A股股票数量
def NumberOfACanBePurchased(All, price):
    #All:资金;DateTime:日期;price:价格
    #A股佣金比例（ACR）：0.0003,不满5元按5元收取;印花税(SD)：向卖方收取0.00005；过户费(TransferF)：0.00001；交易所规费(EF)：0.0000687；
    rA = 0.0003787  #A股交易费用的比例部分
    rA1 = 0.0000787  #A股去除佣金部分的比例系数
    ACRA = 0.0003  #A股佣金比例（ACR）
    ACRFA = 5  #A股佣金固定数值
    nA = All / ((1 + rA) * price)
    if price * nA * ACRA > 5:
        nA = math.floor(nA)
        return nA
    nA = (All - ACRFA) / ((1 + rA1) * price)
    if price * nA * ACRA <= 5:
        if nA < 0:
            return 0
        nA = math.floor(nA)
        return nA


#购买H股函数，返回这笔资金能购买的H股股票数量
def NumberOfHCanBePurchased(All, price, ExchangeRate):
    # 港股佣金比例(ACR)：0.0025，最低收费100港币；证监会交易征费(SFCTL)：0.000027;财务汇报局交易征费(FRCTL)：0.0000015;交易费(TF):0.0000565;H股印花税(SD)：0.0013，不足一元亦作一元计;
    # 转手纸印花税(SDOTP):5港币，卖方负责缴付；过户费用(TransferF):2.50港币，由买方支付
    T = 2.5 * ExchangeRate  #过户费用
    rH = 0.003885  #H股交易费用的比例部分
    rH1 = 0.001385  #H股去除佣金部分的比例系数
    ACRFH = 100 * ExchangeRate  #H股佣金固定数值
    ACRH = 0.0025  #H股佣金比例（ACR）
    nH = (All - T) / ((1 + rH) * price)
    if price * nH * ACRH > 100 * ExchangeRate:
        nH = math.floor(nH)
        return nH
    nH = (All - T - ACRFH) / ((1 + rH1) * price)
    if price * nH * ACRH <= 100 * ExchangeRate:
        if nH < 0:
            return 0
        nH = math.floor(nH)
        return nH


#交易函数，将模拟交易过程,返回为收益率,开仓次数,平仓次数
def trade(AID, All, data, CD, T0, tf, tt, dc, do, ds):
    print("*" * 100)
    print("开始交易", "证券代码:", AID, "开始日期:", T0, "形成期长度:", tf, "交易期长度:", tt)
    # 是否为配对，1为是，0为否;截距;系数;残差的标准差
    print(data)
    par = IfPair(data, T0, tf, CD)
    if par[0] == 0:
        print("不构成配对，交易结束")
        return [0,0,0,0]
    #dc平仓阈值系数,do为开仓阈值系数，ds为止损阈值系数
    dok = par[3] * do
    dck = par[3] * dc
    dsk = par[3] * ds
    T0 = datetime.datetime.strptime(T0, "%Y-%m-%d")
    T1 = T0 + relativedelta(months=tf)
    T2 = T0 + relativedelta(months=tf + tt)
    TradingPeriodData = data.loc[T1:T2]
    openingTimes = 0
    closingTimes = 0
    # A股数量;港股数量
    ExchangeRate0 = HKDCNY['开盘价(元)'][TradingPeriodData.iloc[0].name]
    PA0 = TradingPeriodData.iloc[:, 0][TradingPeriodData.iloc[0].name]
    PH0 = TradingPeriodData.iloc[:, 1][TradingPeriodData.iloc[0].name]
    par1 = AllotmentAmount(All, par[2], ExchangeRate0, PA0, PH0)
    nA = par1[0]
    nAStatic=nA
    nAStart = par1[0]
    nH = par1[1]
    nHStart = par1[1]
    nHStatic=nH
    cash = Decimal(All).quantize(Decimal("0.00")) - Decimal(nA * PA0).quantize(Decimal("0.00")) - CalculateATF(0,
                                                                                                               nA * PA0) - Decimal(
        nH * PH0).quantize(Decimal("0.00")) - CalculateHTF(0, nH * PH0, ExchangeRate0)
    rePre = None
    IsOpen = False
    redf = []  #用来记录每次模拟交易的残差值
    for index, row in TradingPeriodData.iterrows():
        print(index)
        re = math.log(row[1]) - math.log(row[0]) * par[2] - par[1]
        redf.append(re)
        reAbs = abs(re)
        ExchangeRate = HKDCNY['开盘价(元)'][index]
        print(re, re * 100 / par[3], dck, dok, dsk)
        if reAbs > dsk:
            if nA != 0:
                nACash = Decimal(nA * row[0]).quantize(Decimal("0.00")) - CalculateATF(1, nA * row[0])
                nA = 0
                cash = cash + nACash
            if nH != 0:
                nHCash = Decimal(nH * row[1]).quantize(Decimal("0.00")) - CalculateHTF(1, nH * row[1], ExchangeRate)
                nH = 0
                cash = cash + nHCash
            nAStaticCash = Decimal(nAStatic * row[0]).quantize(Decimal("0.00")) - CalculateATF(1, nAStatic * row[0])
            nHStaticCash=Decimal(nHStatic * row[1]).quantize(Decimal("0.00")) - CalculateHTF(1, nHStatic * row[1], ExchangeRate)
            staticCash=nAStaticCash+nHStaticCash
            staticYieldRate=(staticCash-All)/All
            yieldRate = (cash - All) / All
            excessYieldRate=yieldRate-staticYieldRate
            print("超出阈值，停止交易")
            redf.append(yieldRate)
            redf.append(openingTimes)
            redf.append(closingTimes)
            redf.append(excessYieldRate)
            return redf
        if rePre is None:
            if reAbs > dok:
                if re > 0:
                    nHCash = Decimal(nH * row[1]).quantize(Decimal("0.00")) - CalculateHTF(1, nH * row[1], ExchangeRate)
                    nH = 0
                    cash = cash + nHCash
                    nAP = NumberOfACanBePurchased(float(cash), row[0])
                    cash = cash - Decimal(nAP * row[0]).quantize(Decimal("0.00")) - CalculateATF(0, nAP * row[0])
                    nA = nA + nAP
                if re < 0:
                    nACash = Decimal(nA * row[0]).quantize(Decimal("0.00")) - CalculateATF(1, nA * row[0])
                    nA = 0
                    cash = cash + nACash
                    nHP = NumberOfHCanBePurchased(float(cash), row[1], ExchangeRate)
                    cash = cash - Decimal(nHP * row[1]).quantize(Decimal("0.00")) - CalculateHTF(0, nHP * row[1],
                                                                                                 ExchangeRate)
                    nH = nH + nHP
                print("开仓")
                IsOpen = True
                openingTimes = openingTimes + 1
            # if reAbs<=dck:
            #     if nA>nAStart:
            #         deltaA=nA-nAStart
            #         nACash=Decimal(deltaA*row[0]).quantize(Decimal("0.00"))-CalculateATF(1,deltaA*row[0])
            #         cash=cash+nACash
            #         nA=nAStart
            #     if nH>nHStart:
            #         deltaH=nH-nHStart
            #         nHCash=Decimal(deltaH*row[1]).quantize(Decimal("0.00"))-CalculateHTF(1,deltaH*row[1],ExchangeRate)
            #         cash=cash+nHCash
            #         nH=nHStart
            #     if nA<nAStart:
            #         deltaA=nAStart-nA
            #         nA=nAStart
            #         nACash=-1*Decimal(deltaA*row[0]).quantize(Decimal("0.00"))-CalculateATF(0,deltaA*row[0])
            #         cash=cash+nACash
            #     if nH<nHStart:
            #         deltaH=nHStart-nH
            #         nH=nHStart
            #         nHCash=-1*Decimal(deltaH*row[1]).quantize(Decimal("0.00"))-CalculateHTF(0,deltaH*row[1],ExchangeRate)
            #         cash=cash+nHCash
            #     print("平仓")
            #     closingTimes=closingTimes+1
        else:
            if (reAbs > dok and (not IsOpen)) or (reAbs > dok and IsOpen and re * rePre < 0):
                if re > 0:
                    nHCash = Decimal(nH * row[1]).quantize(Decimal("0.00")) - CalculateHTF(1, nH * row[1], ExchangeRate)
                    nH = 0
                    cash = cash + nHCash
                    nAP = NumberOfACanBePurchased(float(cash), row[0])
                    cash = cash - Decimal(nAP * row[0]).quantize(Decimal("0.00")) - CalculateATF(0, nAP * row[0])
                    nA = nA + nAP
                if re < 0:
                    nACash = Decimal(nA * row[0]).quantize(Decimal("0.00")) - CalculateATF(1, nA * row[0])
                    nA = 0
                    cash = cash + nACash
                    nHP = NumberOfHCanBePurchased(float(cash), row[1], ExchangeRate)
                    cash = cash - Decimal(nHP * row[1]).quantize(Decimal("0.00")) - CalculateHTF(0, nHP * row[1],
                                                                                                 ExchangeRate)
                    nH = nH + nHP
                print("开仓")
                IsOpen = True
                openingTimes = openingTimes + 1
            if abs(re / par[3]) <= 0.001 and reAbs < dok:
                if nA > nAStart:
                    deltaA = nA - nAStart
                    nACash = Decimal(deltaA * row[0]).quantize(Decimal("0.00")) - CalculateATF(1, deltaA * row[0])
                    cash = cash + nACash
                    nA = nAStart
                if nH > nHStart:
                    deltaH = nH - nHStart
                    nHCash = Decimal(deltaH * row[1]).quantize(Decimal("0.00")) - CalculateHTF(1, deltaH * row[1],
                                                                                               ExchangeRate)
                    cash = cash + nHCash
                    nH = nHStart
                if nA < nAStart:
                    deltaA = nAStart - nA
                    nA = nAStart
                    nACash = -1 * Decimal(deltaA * row[0]).quantize(Decimal("0.00")) - CalculateATF(0, deltaA * row[0])
                    cash = cash + nACash
                if nH < nHStart:
                    deltaH = nHStart - nH
                    nH = nHStart
                    nHCash = -1 * Decimal(deltaH * row[1]).quantize(Decimal("0.00")) - CalculateHTF(0, deltaH * row[1],
                                                                                                    ExchangeRate)
                    cash = cash + nHCash
                print("平仓")
                IsOpen = False
                closingTimes = closingTimes + 1
        rePre = re
    ExchangeRateLast = HKDCNY['开盘价(元)'][TradingPeriodData.tail(1).index][0]
    PALast = TradingPeriodData.iloc[:, 0][TradingPeriodData.tail(1).index][0]
    PHLast = TradingPeriodData.iloc[:, 1][TradingPeriodData.tail(1).index][0]
    if nA != 0:
        nACash = Decimal(nA * PALast).quantize(Decimal("0.00")) - CalculateATF(1, nA * PALast)
        nA = 0
        cash = cash + nACash
    if nH != 0:
        nHCash = Decimal(nH * PHLast).quantize(Decimal("0.00")) - CalculateHTF(1, nH * PHLast, ExchangeRateLast)
        nH = 0
        cash = cash + nHCash
    nAStaticCash =Decimal(nAStatic * PALast).quantize(Decimal("0.00")) - CalculateATF(1, nAStatic * PALast)
    nHStaticCash=Decimal(nHStatic * PHLast).quantize(Decimal("0.00")) - CalculateHTF(1, nHStatic * PHLast, ExchangeRateLast)
    staticCash=nAStaticCash+nHStaticCash
    staticYieldRate=(staticCash-All)/All
    yieldRate = (cash - All) / All
    excessYieldRate=yieldRate-staticYieldRate
    print("结束交易")
    redf.append(yieldRate)
    redf.append(openingTimes)
    redf.append(closingTimes)
    redf.append(excessYieldRate)
    return redf


#创建文件
def mkdir(path):
    #输入文件路径
    folder = os.path.exists(path)

    if not folder:  #判断是否存在文件夹如果不存在则创建为文件夹
        os.makedirs(path)  #makedirs 创建文件时如果路径不存在会创建这个路径
        print("---  new folder...  ---")
        print("---  OK  ---")

    else:
        print("---  There is this folder!  ---")


# print(HKDCNY)
SHTest = ['601211.SH', '601828.SH', '600958.SH']
SZTest = ['002821.SZ', '000921.SZ', '002490.SZ']


def Main(IDSet, IsSH):
    All = 1000000
    CD = 0.05
    mkdir('..\\result')
    path1 = None
    path2 = None
    path3 = None
    endTime = '2023-08-25'
    endTime = datetime.datetime.strptime(endTime, "%Y-%m-%d")
    if IsSH:
        path1 = "..\\result\SH"
        interInterworkingTime = '2014-11-17'
        interInterworkingTime = datetime.datetime.strptime(interInterworkingTime, "%Y-%m-%d")
    else:
        path1 = "..\\result\SZ"
        interInterworkingTime = '2016-12-05'
        interInterworkingTime = datetime.datetime.strptime(interInterworkingTime, "%Y-%m-%d")
    kli = [0.025, 0.05, 0.075, 0.1, 0.125, 0.15, 0.2, 0.25, 0.5]
    mkdir(path1)
    AllpreOtherDic = {}
    AllpreNameLi = []
    AllpreYieldRateLi = []
    AllpreOpeningTimesLi = []
    AllpreClosingTimesLi = []
    AllpostOtherDic = {}
    AllpostNameLi = []
    AllpostYieldRateLi = []
    AllpostOpeningTimesLi = []
    AllpostClosingTimesLi = []
    AllpreExcessYieldRateLi = []
    AllpostExcessYieldRateLi = []
    AllpreSharpeRateLi=[]
    AllpostSharpeRateLi=[]
    for k in kli:
        path2 = path1 + '\k' + str(k)
        mkdir(path2)
        KpreOtherDic = {}
        KpreNameLi = []
        KpreYieldRateLi = []
        KpreOpeningTimesLi = []
        KpreClosingTimesLi = []
        KpostOtherDic = {}
        KpostNameLi = []
        KpostYieldRateLi = []
        KpostOpeningTimesLi = []
        KpostClosingTimesLi = []
        KpreExcessYieldRateLi = []
        KpostExcessYieldRateLi = []
        KpreSharpeRateLi=[]
        KpostSharpeRateLi=[]
        for tf in range(1, 25):
            for tt in range(1, 13):
                path3 = path2 + '\\tf' + str(tf) + 'tt' + str(tt)
                mkdir(path3)
                path4 = path3 + '\pre'
                mkdir(path4)
                path5 = path3 + '\post'
                mkdir(path5)
                preOtherDic = {}
                preNameLi = []
                preExcessYieldRateLi = []
                preYieldRateLi=[]
                preOpeningTimesLi = []
                preClosingTimesLi = []
                postOtherDic = {}
                postNameLi = []
                postYieldRateLi = []
                postExcessYieldRateLi = []
                postOpeningTimesLi = []
                postClosingTimesLi = []
                preSharpeRateLi=[]
                postSharpeRateLi=[]
                for AID in IDSet:
                    data = DataSelect(AID)
                    data = DataClean(data)
                    T0Li = []
                    reDIC = {}
                    savepath = ''
                    for i in range(0, data.shape[0]):
                        T0 = data.iloc[i].name
                        T0 = T0.replace(day=1)
                        for T01 in T0Li:
                            if T01 == T0:
                                break
                        T0Li.append(T0)
                    preTempYieldRateLi = []
                    preTempOpeningTimesLi = []
                    preTempClosingTimesLi = []
                    postTempYieldRateLi = []
                    postTempOpeningTimesLi = []
                    postTempClosingTimesLi = []
                    preTempExcessYieldRateLi=[]
                    postTempExcessYieldRateLi=[]
                    for T0 in T0Li:
                        if (T0 +relativedelta(months=tf + tt) > interInterworkingTime and T0 < interInterworkingTime) or (
                                T0 > interInterworkingTime and (T0+relativedelta(months=tf + tt) > endTime)):
                            break
                        # [yieldRate, openingTimes, closingTimes,redf]
                        reliName = str(T0) + 'tf' + str(tf) + 'tt' + str(tt)
                        T0Copy=T0.strftime("%Y-%m-%d")
                        dataCopy=data.copy()
                        tradeData = trade(AID, All, dataCopy, CD, T0Copy, tf, tt, 0, k, 1.96)
                        reDIC.update({reliName: tradeData})
                        if T0 < interInterworkingTime:
                            preTempYieldRateLi.append(tradeData[-4])
                            preTempOpeningTimesLi.append(tradeData[-3])
                            preTempClosingTimesLi.append(tradeData[-2])
                            preTempExcessYieldRateLi.append(tradeData[-1])
                            savepath = path4
                        else:
                            preTempYieldRateLi.append(tradeData[-4])
                            postTempOpeningTimesLi.append(tradeData[-3])
                            postTempClosingTimesLi.append(tradeData[-2])
                            postTempExcessYieldRateLi.append(tradeData[-1])
                            savepath = path5
                    preNameLi.append(AID)
                    preYieldRateLi.append(np.mean(preTempYieldRateLi))
                    preExcessYieldRateLi.append(np.mean(preTempExcessYieldRateLi))
                    preOpeningTimesLi.append(np.mean(preTempOpeningTimesLi))
                    preClosingTimesLi.append(np.mean(preTempClosingTimesLi))
                    print(preTempExcessYieldRateLi)
                    print(preTempYieldRateLi)
                    preSharpeRateLi.append(np.mean(preTempExcessYieldRateLi)/np.std(preTempYieldRateLi))
                    postNameLi.append(AID)
                    postYieldRateLi.append(np.mean(postTempYieldRateLi))
                    postOpeningTimesLi.append(np.mean(postTempOpeningTimesLi))
                    postClosingTimesLi.append(np.mean(postTempClosingTimesLi))
                    postExcessYieldRateLi.append(np.mean(postTempExcessYieldRateLi))
                    postSharpeRateLi.append(np.mean(postTempExcessYieldRateLi)/np.std(postTempYieldRateLi))
                    reDataOut = pd.DataFrame.from_dict(reDIC)
                    reDataOut.to_csv(savepath + '\\' + AID + '.csv')
                KpreNameLi.append('tf'+str(tf)+'tt'+str(tt))
                KpostNameLi.append('tf'+str(tf)+'tt'+str(tt))
                KpreYieldRateLi.append(np.mean(preYieldRateLi))
                KpreOpeningTimesLi.append(np.mean(preOpeningTimesLi))
                KpreClosingTimesLi.append(np.mean(preClosingTimesLi))
                KpreExcessYieldRateLi.append(np.mean(preExcessYieldRateLi))
                KpreSharpeRateLi.append(np.mean(preExcessYieldRateLi)/np.std(preYieldRateLi))
                KpostYieldRateLi.append(np.mean(postYieldRateLi))
                KpostOpeningTimesLi.append(np.mean(postOpeningTimesLi))
                KpostClosingTimesLi.append(np.mean(postClosingTimesLi))
                KpostExcessYieldRateLi.append(np.mean(postExcessYieldRateLi))
                KpostSharpeRateLi.append(np.mean(postExcessYieldRateLi)/np.std(postYieldRateLi))
                preOtherDic.update({'AID':preNameLi,'YieldRate':preYieldRateLi,'OpeningTimes':preOpeningTimesLi,'ClosingTimes':preClosingTimesLi,'ExcessYieldRate':preExcessYieldRateLi,'SharpeRate':preSharpeRateLi})
                postOtherDic.update({'AID':postNameLi,'YieldRate':postYieldRateLi,'OpeningTimes':postOpeningTimesLi,'ClosingTimes':postClosingTimesLi,'ExcessYieldRate':postExcessYieldRateLi,'SharpeRate':postSharpeRateLi})
                preOtherDataOut=pd.DataFrame.from_dict(preOtherDic)
                postOtherDataOut=pd.DataFrame.from_dict(postOtherDic)
                preOtherDataOut.to_csv(path3+'\\'+'PreSummary.csv')
                postOtherDataOut.to_csv(path3+'\\'+'PostSummary.csv')
        KpreOtherDic.update({'TimePair':KpreNameLi,'YieldRate':KpreYieldRateLi,'OpeningTimes':KpreOpeningTimesLi,'ClosingTimes':KpreClosingTimesLi,'ExcessYieldRate':KpreExcessYieldRateLi,'SharpeRate':KpreSharpeRateLi})
        KpostOtherDic.update({'TimePair':KpostNameLi,'YieldRate':KpostYieldRateLi,'OpeningTimes':KpostOpeningTimesLi,'ClosingTimes':KpostClosingTimesLi,'ExcessYieldRate':KpostExcessYieldRateLi,'SharpeRate':KpostSharpeRateLi})
        KpreOtherDic.to_csv(path2+'\\'+'PreSummary.csv')
        KpostOtherDic.to_csv(path2+'\\'+'PostSummary.csv')
        AllpreNameLi.append('k'+str(k))
        AllpostNameLi.append('k'+str(k))
        AllpreYieldRateLi.append(np.mean(KpreYieldRateLi))
        AllpreOpeningTimesLi.append(np.mean(KpreOpeningTimesLi))
        AllpreClosingTimesLi.append(np.mean(KpreClosingTimesLi))
        AllpreExcessYieldRateLi.append(np.mean(KpreExcessYieldRateLi))
        AllpreSharpeRateLi.append(np.mean(KpreExcessYieldRateLi)/np.std(KpreYieldRateLi))
        AllpostYieldRateLi.append(np.mean(KpostYieldRateLi))
        AllpostOpeningTimesLi.append(np.mean(KpostOpeningTimesLi))
        AllpostClosingTimesLi.append(np.mean(KpostClosingTimesLi))
        AllpostExcessYieldRateLi.append(np.mean(KpostExcessYieldRateLi))
        AllpostSharpeRateLi.append(np.mean(KpostExcessYieldRateLi)/np.std(KpostYieldRateLi))
    AllpreOtherDic.update({'KValue':AllpreNameLi,'YieldRate':AllpreYieldRateLi,'OpeningTimes':AllpreOpeningTimesLi,'ClosingTimes':AllpreClosingTimesLi,'ExcessYieldRate':AllpreExcessYieldRateLi,'SharpeRate':AllpreSharpeRateLi})
    AllpostOtherDic.update({'KValue':AllpostNameLi,'YieldRate':AllpostYieldRateLi,'OpeningTimes':AllpostOpeningTimesLi,'ClosingTimes':AllpostClosingTimesLi,'ExcessYieldRate':AllpreExcessYieldRateLi,'SharpeRate':AllpreSharpeRateLi})
    AllpreOtherDic.to_csv(path1+'\\'+'PreSummary.csv')
    AllpostOtherDic.to_csv(path1+'\\'+'PostSummary.csv')
    # AID = "601211.SH"
    #
    # T0 = '2016-01-01'
    # tf = 30
    # tt = 1
    # All = 1000000
    # li = trade(AID, All, data, CD, T0, tf, tt, 0, 0.1, 1.96)
    # try:
    #     print("收益率:", li[0], "开仓次数:", li[1], "平仓次数:", li[2])
    # except:
    #     print(li)


# AID = "601828.SH"
# data = DataSelect(AID)
# data = DataClean(data)
# print(data)
# T0=data.iloc[0].name
# print(T0)
Main(SHTest, 1)
# print(s3[s3.index=="2023-8-20"])
# s3.to_csv("1.csv")

---  new folder...  ---
---  OK  ---
---  new folder...  ---
---  OK  ---
---  new folder...  ---
---  OK  ---
---  new folder...  ---
---  OK  ---
---  new folder...  ---
---  OK  ---
---  new folder...  ---
---  OK  ---
****************************************************************************************************
开始交易 证券代码: 601211.SH 开始日期: 2017-04-01 形成期长度: 1 交易期长度: 1
            601211.SH  02611.HK
date                           
2017-04-11    19.5196   14.0570
2017-04-12    19.5619   14.0528
2017-04-13    19.5936   13.9929
2017-04-18    19.2556   14.0304
2017-04-19    19.0338   13.9732
...               ...       ...
2023-08-21    19.2496   11.3881
2023-08-22    18.9710   11.2063
2023-08-23    18.9203   11.2433
2023-08-24    18.9836   11.2504
2023-08-25    18.8823   11.3067

[1509 rows x 2 columns]
date
2017-04-11    0.002014
2017-04-12    0.001528
2017-04-13   -0.002883
2017-04-18    0.001296
2017-04-19   -0.001789
2017-04-20   -0.002513
2017-04-21    0.002574
2017-04-24   -

d:\py\lib\site-packages\pandas\core\indexing.py:1951: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[selected_item_labels] = value
d:\py\lib\site-packages\pandas\core\indexing.py:1951: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[selected_item_labels] = value
d:\py\lib\site-packages\pandas\core\indexing.py:1951: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the document

            601211.SH  02611.HK
date                           
2017-04-11    19.5196   14.0570
2017-04-12    19.5619   14.0528
2017-04-13    19.5936   13.9929
2017-04-18    19.2556   14.0304
2017-04-19    19.0338   13.9732
...               ...       ...
2023-08-21    19.2496   11.3881
2023-08-22    18.9710   11.2063
2023-08-23    18.9203   11.2433
2023-08-24    18.9836   11.2504
2023-08-25    18.8823   11.3067

[1509 rows x 2 columns]
date
2017-04-11    0.002014
2017-04-12    0.001528
2017-04-13   -0.002883
2017-04-18    0.001296
2017-04-19   -0.001789
2017-04-20   -0.002513
2017-04-21    0.002574
2017-04-24   -0.001733
2017-04-25   -0.002905
2017-04-26    0.001682
2017-04-27    0.001198
2017-04-28    0.001530
dtype: float64
2017-05-02 00:00:00
0.003573751620173038 166.67504447452842 0.0 5.3603579819640985e-05 0.004202520657859853
开仓
2017-05-04 00:00:00
-0.006670652484836559 -311.11040098819825 0.0 5.3603579819640985e-05 0.004202520657859853
超出阈值，停止交易
********************************

d:\py\lib\site-packages\pandas\core\indexing.py:1951: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[selected_item_labels] = value
d:\py\lib\site-packages\pandas\core\indexing.py:1951: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[selected_item_labels] = value
d:\py\lib\site-packages\pandas\core\indexing.py:1951: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the document

2017-05-02 00:00:00
0.003573751620173038 166.67504447452842 0.0 5.3603579819640985e-05 0.004202520657859853
开仓
2017-05-04 00:00:00
-0.006670652484836559 -311.11040098819825 0.0 5.3603579819640985e-05 0.004202520657859853
超出阈值，停止交易
****************************************************************************************************
开始交易 证券代码: 601211.SH 开始日期: 2017-05-01 形成期长度: 1 交易期长度: 1
            601211.SH  02611.HK
date                           
2017-04-11    19.5196   14.0570
2017-04-12    19.5619   14.0528
2017-04-13    19.5936   13.9929
2017-04-18    19.2556   14.0304
2017-04-19    19.0338   13.9732
...               ...       ...
2023-08-21    19.2496   11.3881
2023-08-22    18.9710   11.2063
2023-08-23    18.9203   11.2433
2023-08-24    18.9836   11.2504
2023-08-25    18.8823   11.3067

[1509 rows x 2 columns]
date
2017-05-02    0.029940
2017-05-04    0.020245
2017-05-05    0.018008
2017-05-08    0.009443
2017-05-09   -0.009777
2017-05-10   -0.018525
2017-05-11   -0.000842
2017-

...

d:\py\lib\site-packages\pandas\core\indexing.py:1951: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[selected_item_labels] = value
d:\py\lib\site-packages\pandas\core\indexing.py:1951: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[selected_item_labels] = value
d:\py\lib\site-packages\pandas\core\indexing.py:1951: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the document

date
2023-06-01   -0.015923
2023-06-02   -0.010842
2023-06-05   -0.016056
2023-06-06   -0.012223
2023-06-07   -0.002068
2023-06-08   -0.015099
2023-06-09    0.008321
2023-06-12   -0.005979
2023-06-13    0.007478
2023-06-14    0.007510
2023-06-15    0.004609
2023-06-16    0.010700
2023-06-19    0.001448
2023-06-20    0.020815
2023-06-21    0.017376
2023-06-26   -0.017187
2023-06-27   -0.000785
2023-06-28    0.004197
2023-06-29    0.006642
2023-06-30    0.007066
dtype: float64
2023-07-03 00:00:00
0.01774103229917179 154.40037063717506 0.0 0.0002872569577708687 0.022520945489236107
开仓
2023-07-04 00:00:00
0.019224763585008464 167.313297945799 0.0 0.0002872569577708687 0.022520945489236107
2023-07-05 00:00:00
0.01783682257122443 155.2340342740455 0.0 0.0002872569577708687 0.022520945489236107
2023-07-06 00:00:00
0.019194155596155005 167.046916331486 0.0 0.0002872569577708687 0.022520945489236107
2023-07-07 00:00:00
0.005318105796665584 46.283524670162954 0.0 0.0002872569577708687 0.02252094

d:\py\lib\site-packages\pandas\core\indexing.py:1951: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[selected_item_labels] = value
d:\py\lib\site-packages\pandas\core\indexing.py:1951: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[selected_item_labels] = value
d:\py\lib\site-packages\pandas\core\indexing.py:1951: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the document

2023-07-03 00:00:00
0.01774103229917179 154.40037063717506 0.0 0.0002872569577708687 0.022520945489236107
开仓
2023-07-04 00:00:00
0.019224763585008464 167.313297945799 0.0 0.0002872569577708687 0.022520945489236107
2023-07-05 00:00:00
0.01783682257122443 155.2340342740455 0.0 0.0002872569577708687 0.022520945489236107
2023-07-06 00:00:00
0.019194155596155005 167.046916331486 0.0 0.0002872569577708687 0.022520945489236107
2023-07-07 00:00:00
0.005318105796665584 46.283524670162954 0.0 0.0002872569577708687 0.022520945489236107
2023-07-10 00:00:00
0.003776501703219637 32.86692977365559 0.0 0.0002872569577708687 0.022520945489236107
2023-07-11 00:00:00
0.0018761959365729552 16.32855084810085 0.0 0.0002872569577708687 0.022520945489236107
2023-07-12 00:00:00
0.003428853289063838 29.84134236183473 0.0 0.0002872569577708687 0.022520945489236107
2023-07-13 00:00:00
0.0022504532281888423 19.585715570238012 0.0 0.0002872569577708687 0.022520945489236107
2023-07-14 00:00:00
0.00387383205014058 33

TypeError: unsupported operand type(s) for /: 'float' and 'decimal.Decimal'

In [80]:

# print(os.path.relpath('F:\热爱学习\jupyter\量化交易\配对交易\\result', "F:\热爱学习\jupyter\量化交易\配对交易\程序"))
# print(AllotmentAmount(100000, IfPair(data, T0, tf, CD)[2], HKDCNY['开盘价(元)'][DateTime], data.iloc[:, 0][DateTime],data.iloc[:, 1][DateTime]))
# print(NumberOfHCanBePurchased(100000, data.iloc[:, 1][DateTime], HKDCNY['开盘价(元)'][DateTime]))
# print(CalculateHTF(False, 1000000, DateTime))
# print(CalculateATF(True, 1000))
# print(IfPair(data,start,length,0.05))
# re=y-results.params[0]-results.params[1]*x
# print(data['2017-04-11':'2017-04-15'])